In [38]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
file_path = '/home/geonpeter/Desktop/mydisk/Big Data/Projects/AzureBigData/dataset/olist_order_payments_dataset.csv'

In [ ]:
df = pd.read_csv(file_path, header= False)
df.shape

(103886, 5)

In [31]:
table_name = 'olist_order_payments'

In [ ]:
try:
    connection = mysql.connector.connect(
        host = os.getenv('host'),
        database = os.getenv('database'),
        port = os.getenv('port'),
        user = os.getenv('user'),
        password = os.getenv('password')
    )
    batch_size = 500
    create_query = f'CREATE TABLE IF NOT EXISTS {table_name}(order_id VARCHAR(50), payment_sequential INT, payment_type VARCHAR(50), payment_installments INT, payment_value FLOAT);'
    insert_query = f'INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s);'
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute(create_query)
        df = pd.read_csv(file_path)
        cursor.execute('START TRANSACTION;')
        for start in range(0,len(df),batch_size):
            end = start + batch_size
            batch = df.iloc[start:end]
            batch_recs = [tuple(row) for row in batch.itertuples(index=False, name=None)]
            cursor.executemany(insert_query, batch_recs)
            print(f'batch {start+1} - {end} loaded')
        connection.commit()
        print('Data Loaded Successfully')
            
except Error as e:
    print('Error ',e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print('Connection Closed')

Error  1226 (42000): User 'dbecom_caughtbush' has exceeded the 'max_user_connections' resource (current value: 5)
